In [1]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import datetime
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
storage = "/Volumes/easystore/Drones"

cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/geocodio/all-flights-manifest-extra-col_geocodio.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [2]:
cv_flights = cv_flights[cv_flights['department']=='cvpd'].copy()
cv_flights["drone-date"] = pd.to_datetime(cv_flights["date"]).dt.date



In [3]:
cv_flights["match"] = cv_flights["incident_id"].astype(str).apply(lambda x: x.strip().replace(' ','').replace('L','0').replace('l','0')[-5:])

cv_flights["drone-date"] =  pd.to_datetime(cv_flights['date'] + " " + cv_flights['time_s'])

calls_for_service["match"] = (
    calls_for_service["Incident No."].astype(str).apply(lambda x: x.strip().replace(' ','').replace('l','0').replace('L','0')[-5:])
)
calls_for_service["cfs-date"] = pd.to_datetime(calls_for_service["Date"])

In [4]:
matches = cv_flights[cv_flights["match"].isin(calls_for_service["match"])]

In [5]:
matches_joined_cfs = pd.merge(matches, calls_for_service, how="left", on="match",suffixes=['-drones','-cfs'])


In [6]:

matches_joined_cfs = matches_joined_cfs[
    matches_joined_cfs["drone-date"].dt.date == matches_joined_cfs["cfs-date"].dt.date
]
flights_with_cfs  = matches_joined_cfs.copy()



In [13]:
flights_with_cfs.to_csv('../../data/outputs/outputs_drones-with-matching-cfs_matched-by-ID.csv',index=False)

In [14]:
no_matching_cfs = cv_flights[~cv_flights['id'].isin(flights_with_cfs['id'])].copy()
no_matching_cfs = no_matching_cfs[no_matching_cfs['drone-date'].dt.date >= pd.to_datetime('2021-07-01')].copy()

In [15]:
no_matching_cfs.to_csv('../../data/outputs/outputs_drones-without-matching-cfs_matched-by-ID.csv',index=False)

In [11]:
flights_with_cfs.shape[0]

1953

# extra

In [298]:
no_cfs = []
for _, row in tqdm(no_matching_cfs.sample(frac=1).iterrows()):
    add_twenty = row['drone-date'] + datetime.timedelta(minutes = 20)
    minus_twenty = row['drone-date'] - datetime.timedelta(minutes = 20)

    mask = (calls_for_service['cfs-date'] > minus_twenty) & (calls_for_service['cfs-date'] < add_twenty)
    possible_calls = calls_for_service.loc[mask].copy()
    possible_calls['drone-id'] = row['id']
    possible_calls['drone-incident-id'] = row['incident_id']
    possible_calls['drone-date'] = row['drone-date']
    possible_calls['drone-type'] = row['type']
    possible_calls['drone-type'] = row['type']
    possible_calls['drone-address'] = row['address_map']
    possible_calls['drone-FIPS'] = row['Full FIPS (block)']

    # possible_calls = possible_calls[possible_calls['Full FIPS (block)']==row['Full FIPS (block)']].copy()
    
    no_cfs.append(possible_calls[['Incident No.','drone-incident-id','Description','drone-type','cfs-date', 'drone-date','Block Location','drone-address','Full FIPS (block)','drone-FIPS','drone-id']])
    

0it [00:00, ?it/s]

In [301]:
posible_calls = pd.concat(no_cfs)

In [302]:
posible_calls.loc[posible_calls['Full FIPS (block)'] == posible_calls['drone-FIPS'],'FIPS_MATCH'] = True
posible_calls.loc[posible_calls['Full FIPS (block)'] != posible_calls['drone-FIPS'],'FIPS_MATCH'] = False


In [304]:

posible_calls.to_csv('./possible-cfs.csv',index=False)

In [306]:
no_matching_cfs['id'].to_csv('./missing-cfs-flights.csv',index=False)

In [163]:
add_twenty = row['drone-date'] + datetime.timedelta(minutes = 20)
minus_twenty = row['drone-date'] - datetime.timedelta(minutes = 20)

mask = (calls_for_service['cfs-date'] > minus_twenty) & (calls_for_service['cfs-date'] < add_twenty)
calls_for_service.loc[mask]

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,match,cfs-date
37543,230731L00064792,2023-07-31 10:58:28.910,Municipal Code Violation,3,HANDLED NO REPORT,citizen,91911,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64792,2023-07-31 10:58:28.910
37544,230731L00064791,2023-07-31 11:01:41.297,Grand Theft > $950,5,REPORT TAKEN,citizen,91913,NaN,2310285.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64791,2023-07-31 11:01:41.297
37545,230731L00064796,2023-07-31 11:02:16.433,Disturbance - General,3,HANDLED NO REPORT,citizen,91911,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64796,2023-07-31 11:02:16.433
37546,230731L00064794,2023-07-31 11:02:53.283,Lost Property,5,REPORT TAKEN,officer,91913,NaN,2310273.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64794,2023-07-31 11:02:53.283
37547,230731L00064795,2023-07-31 11:03:11.597,Vehicle Burglary,5,REPORT TAKEN,officer,91915,NaN,2310275.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64795,2023-07-31 11:03:11.597
37548,230731L00064797,2023-07-31 11:04:14.013,NaN,7,HANDLED NO REPORT,citizen,91913,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64797,2023-07-31 11:04:14.013
37549,230731L00064798,2023-07-31 11:08:47.023,Extra Patrol,5,HANDLED NO REPORT,officer,91914,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64798,2023-07-31 11:08:47.023
37550,230731L00064799,2023-07-31 11:10:00.403,Security Check,3,HANDLED NO REPORT,officer,91911,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64799,2023-07-31 11:10:00.403
37551,230731L00064801,2023-07-31 11:15:25.427,Illegal Dumping,5,HANDLED NO REPORT,citizen,91911,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64801,2023-07-31 11:15:25.427
37552,230731L00064804,2023-07-31 11:19:00.037,[Protected Call Type],2,ASSISTED,citizen,91910,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64804,2023-07-31 11:19:00.037


In [150]:
row['drone-date'].date()

datetime.date(2021, 4, 30)

In [148]:
calls_for_service[calls_for_service['cfs-date'].dt.date == row['drone-date'].date()]

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,match,cfs-date


In [10]:
calls_for_service['date'] = pd.to_datetime(calls_for_service['Date'])  
mask = (calls_for_service['date'] > '2023-03-06 19:00') & (calls_for_service['date'] < '2023-03-07 01:00')
calls_for_service.loc[mask].sort_values('date')

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,match,cfs-date,date
170872,230306L00019689,2023-03-06 19:08:54.050,Traffic Stop,5,CITATION,officer,91911,900 BLOCK CLUB VIEW TE,NaN,"900 BLOCK CLUB VIEW TE, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,19689,2023-03-06,2023-03-06 19:08:54.050
179334,230306L00019691,2023-03-06 19:32:30.103,Traffic Stop,5,HANDLED NO REPORT,officer,91914,800 BLOCK LANE AV,NaN,"800 BLOCK LANE AV, Chula Vista, CA, 91914",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,19691,2023-03-06,2023-03-06 19:32:30.103
141856,230306L00019692,2023-03-06 19:34:50.280,Extra Patrol,5,HANDLED NO REPORT,officer,91911,1400 BLOCK THIRD AV,NaN,"1400 BLOCK THIRD AV, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,19692,2023-03-06,2023-03-06 19:34:50.280
44683,230306L00019694,2023-03-06 19:43:02.553,Suspicious Circumstances,3,HANDLED NO REPORT,citizen,91911,1100 BLOCK BROADWAY,NaN,"1100 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,19694,2023-03-06,2023-03-06 19:43:02.553
147632,230306L00019693,2023-03-06 19:43:21.857,PR Contact,4,HANDLED NO REPORT,citizen,91915,1700 BLOCK METRO AV,NaN,"1700 BLOCK METRO AV, Chula Vista, CA, 91915",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,19693,2023-03-06,2023-03-06 19:43:21.857
99811,230306L00019696,2023-03-06 20:06:44.007,Graffiti,4,HANDLED NO REPORT,citizen,91910,900 BLOCK MARINA WY,NaN,"900 BLOCK MARINA WY, Chula Vista, CA, 91910",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,19696,2023-03-06,2023-03-06 20:06:44.007
111765,230306L00019697,2023-03-06 20:07:39.110,Suspicious Circumstances,3,HANDLED NO REPORT,citizen,91910,700 BLOCK PLAZA CT,NaN,"700 BLOCK PLAZA CT, Chula Vista, CA, 91910",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,19697,2023-03-06,2023-03-06 20:07:39.110
74916,230306L00019698,2023-03-06 20:11:34.920,Person Down,2,HANDLED NO REPORT,citizen,91911,1300 BLOCK THIRD AV,NaN,"1300 BLOCK THIRD AV, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,19698,2023-03-06,2023-03-06 20:11:34.920
72479,230306L00019700,2023-03-06 20:20:44.103,False Alarm - Undefined,3,508 Left,citizen,91911,500 BLOCK TELEGRAPH CANYON RD,NaN,"500 BLOCK TELEGRAPH CANYON RD, Chula Vista, CA...",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,19700,2023-03-06,2023-03-06 20:20:44.103
86762,230306L00019703,2023-03-06 20:22:43.380,Miscellaneous Assistance,3,HANDLED NO REPORT,citizen,91902,4400 BLOCK BONITA RD,NaN,"4400 BLOCK BONITA RD, Chula Vista, CA, 91902",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,19703,2023-03-06,2023-03-06 20:22:43.380


In [11]:
calls_for_service[L05683

,Description
0,Extra Patrol
1,Narcotics
2,Pedestrian Stop
3,Disturbance - General
5,Pedestrian Stop (Person in Parked Veh)
...,...
41615,SRO Attendance Review Board
46831,SRO Staff Presentation
52602,SRO Property Crime Investigation
102243,SRO Custody


In [16]:
flight_data['type'].str.lower().drop_duplicates()

0                                        traffic collision
1                                             subject down
2                                            lude activity
3                                        domestic violence
5                                            welfare check
                               ...                        
10893                disorderly person / disorderly person
10914                                    person in roadway
10918    burglary alarm / disorderly person / welfare c...
10920                   welfare check / mental health call
10946                                      theft & warrant
Name: type, Length: 989, dtype: object

In [16]:
# L048211
a = calls_for_service[calls_for_service['Block Location'].astype(str).str.contains('1700 block broadway',case=False)]
a.sort_values('date')


,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,match,cfs-date,date
108980,210705L00055706,2021-07-05 17:14:45.977,Petty Theft <=$950,3,ASSISTED,citizen,91911,1700 BLOCK BROADWAY,NaN,"1700 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,55706,2021-07-05,2021-07-05 17:14:45.977
108981,210706L00055801,2021-07-06 01:17:14.397,Ensure Safety of Fire and Medical Personnel,2,ASSISTED,citizen,91911,1700 BLOCK BROADWAY,NaN,"1700 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,55801,2021-07-06,2021-07-06 01:17:14.397
108982,210712L00057984,2021-07-12 21:10:14.260,[Protected Call Type],3,HANDLED NO REPORT,citizen,91911,1700 BLOCK BROADWAY,NaN,"1700 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,57984,2021-07-12,2021-07-12 21:10:14.260
108983,210713L00058206,2021-07-13 15:03:09.090,Vehicle Theft,3,REPORT TAKEN,citizen,91911,1700 BLOCK BROADWAY,2108763.0,"1700 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,58206,2021-07-13,2021-07-13 15:03:09.090
108984,210714L00058513,2021-07-14 17:07:06.287,Vandalism,5,HANDLED NO REPORT,citizen,91911,1700 BLOCK BROADWAY,NaN,"1700 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,58513,2021-07-14,2021-07-14 17:07:06.287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109173,230817L00070320,2023-08-17 14:18:48.463,Petty Theft <=$950,3,HANDLED NO REPORT,citizen,91911,1700 BLOCK BROADWAY,NaN,"1700 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,70320,2023-08-17,2023-08-17 14:18:48.463
109174,230822L00071768,2023-08-22 15:12:01.630,Disturbance - General,3,HANDLED NO REPORT,citizen,91911,1700 BLOCK BROADWAY,NaN,"1700 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,71768,2023-08-22,2023-08-22 15:12:01.630
109175,230825L00072727,2023-08-25 15:01:03.687,Abandoned Vehicle,5,ASSISTED,officer,91911,1700 BLOCK BROADWAY,NaN,"1700 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,72727,2023-08-25,2023-08-25 15:01:03.687
109176,230829L00073955,2023-08-29 11:34:06.833,Disturbance - General,3,HANDLED NO REPORT,citizen,91911,1700 BLOCK BROADWAY,NaN,"1700 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,73955,2023-08-29,2023-08-29 11:34:06.833


In [1]:
flight_data[flight_data['id']=='2228aea6af42ec92d883b594bfee28c1']

NameError: name 'flight_data' is not defined

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,match,cfs-date,date
6382,210703L00054800,2021-07-03 00:03:08.790,Traffic Stop,5,HANDLED NO REPORT,officer,91910,300 BLOCK BROADWAY,NaN,"300 BLOCK BROADWAY, Chula Vista, CA, 91910",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,54800,2021-07-03,2021-07-03 00:03:08.790
82947,210703L00054801,2021-07-03 00:03:32.763,Traffic Stop,5,CITATION,officer,91911,VISTA WY & NAPLES ST,NaN,"VISTA WY & NAPLES ST, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,54801,2021-07-03,2021-07-03 00:03:32.763
82949,210703L00054802,2021-07-03 00:07:10.247,Disturbance - General,2,ASSISTED,citizen,91910,400 BLOCK THIRD AV,NaN,"400 BLOCK THIRD AV, Chula Vista, CA, 91910",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,54802,2021-07-03,2021-07-03 00:07:10.247
83290,210703L00054803,2021-07-03 00:09:00.463,False Alarm - Undefined,3,508 Left,citizen,91911,1600 BLOCK BROADWAY,NaN,"1600 BLOCK BROADWAY, Chula Vista, CA, 91911",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,54803,2021-07-03,2021-07-03 00:09:00.463
2289,210703L00054804,2021-07-03 00:09:41.543,Check a Person's Well Being,3,HANDLED NO REPORT,citizen,91910,300 BLOCK FOURTH AV,NaN,"300 BLOCK FOURTH AV, Chula Vista, CA, 91910",...,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,54804,2021-07-03,2021-07-03 00:09:41.543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29917,230629L00054896,2023-06-29 19:07:03.763,Traffic Stop,5,CITATION,officer,91913,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54896,2023-06-29,2023-06-29 19:07:03.763
29918,230629L00054897,2023-06-29 19:08:25.600,Minor Injury Collision,2,HANDLED NO REPORT,citizen,91913,NaN,2308814.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54897,2023-06-29,2023-06-29 19:08:25.600
29919,230629L00054898,2023-06-29 19:12:20.847,Disturbance - General,3,HANDLED NO REPORT,citizen,91911,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54898,2023-06-29,2023-06-29 19:12:20.847
29920,230629L00054899,2023-06-29 19:20:17.913,Traffic Stop,5,CITATION,officer,91913,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54899,2023-06-29,2023-06-29 19:20:17.913


In [13]:
flight_data['date'] = flight_data['date'].astype(str)
def compile_fl(df):


    df['fn'] = df['id'].apply(lambda x: f"/Volumes/easystore/Drones/flights/kml/{x}.kml")

    compiled_flights = []
    for _, row in df.iterrows():
        d = gpd.read_file(row['fn'], driver='KML')


        d['id'] = row['id']
        d['incident_type'] = row['type']
        d['incident_id'] = row['incident_id']
        d['address_map'] = row['address_map']
        d['date'] = row['date']
        d['time'] = row['time_s']

        compiled_flights.append(d)
    cf = pd.concat(compiled_flights,ignore_index=True)
    cf = gpd.GeoDataFrame(cf)
    cf['incident_type'] = cf['incident_type'].astype(str)
    return cf

In [14]:
c = compile_fl(flight_data[flight_data['date']=='4-01-22'])

/var/folders/nq/51nwndf52yqcj9x8h10zjpg80000gn/T/ipykernel_1219/1164878428.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fn'] = df['id'].apply(lambda x: f"/Volumes/easystore/Drones/flights/kml/{x}.kml")


In [15]:
c[['geometry','incident_type','id']].to_file("../data/7-11.geojson", driver='GeoJSON')

In [ ]:
#sus flights
#a62bb9d5a95f18d60489907be30ec377
#32b810ed673fd7a88f25d28e6ca3fb82
#8a83235cb80e675ed62e3334db29c274
#1de4af6cb9bc8fb54624f44064fb9c93